In [ ]:
from tools import path_from_root
import pandas as pd

df = pd.read_csv(path_from_root("props_vectors.csv"))
ficzers = df.columns.drop("kind")

X = df.loc[:, ficzers].astype("float64")
y = df["kind"]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

le = LabelEncoder()
y_int = le.fit_transform(y).reshape(-1,1)

ohe = OneHotEncoder()
y_onehot = ohe.fit_transform(y_int)

In [ ]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=TEST_SIZE)

In [ ]:
from keras import layers, models

model = models.Sequential()
model.add(layers.Input(shape=(X.shape[1],)))
model.add(layers.Dense(10, "sigmoid"))
model.add(layers.Dense(len(le.classes_), "softmax"))

model.compile(loss="categorical_crossentropy",
              optimizer="sgd", metrics=["accuracy"])

model.summary()

model.fit(X_train, y_train, epochs=100, batch_size=10, shuffle=True)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)